In [ ]:
from pathlib import Path

path = Path("./data")


[PosixPath('data/dara_pose_r_cam_u_lincheng'), PosixPath('data/data_pose_r_cam_u_lincheng2'), PosixPath('data/data_pose_r_cam_u_3'), PosixPath('data/data_pose_r_cam_u_1'), PosixPath('data/data_ss_2'), PosixPath('data/data_pose_r_cam_u_lincheng'), PosixPath('data/data_ss_'), PosixPath('data/data_pose_r_cam_u_lin'), PosixPath('data/data_pose_r_cam_u_2'), PosixPath('data/dara_pose_r_cam_u_lin'), PosixPath('data/resources'), PosixPath('data/data_pose_r_cam_u_4'), PosixPath('data/data_pose_r_cam_u_lincheng3')]


In [13]:
import json
from pathlib import Path
from collections import OrderedDict

file_contents = {}
for json_f in path.rglob("sa*.json"):
    json_contents = json.loads(json_f.read_text())
    print(len(json_contents))
    file_contents.update(json_contents)
    # break

# Sort the file_contents by the 'prompt' field
sorted_contents = OrderedDict(
    sorted(file_contents.items(), key=lambda item: int(item[1]['prompt']))
)

output = Path("./data") / "lincheng"
output.mkdir(exist_ok=True)
with open(output / "saved_data.json", "w") as f:
    json.dump(sorted_contents, f, indent=4)


11
12
15
6
1
1
81
10
10
13
2


In [1]:
import pyperclip

pyperclip.paste()

'Image_Amount'